In [10]:
import numpy as np
import math
from collections import defaultdict

jointCode = {0: 'Head',
             1: 'Neck',
             2: 'Right_Shoulder',
             3: 'Right_Elbow',
             4: 'Right_Wrist',
             5:'Left_Shoulder',
             6:'Left_Elbow',
             7:'Left_Wrist',
             8:'Right_Hip',
             9:'Right_Knee',
             10:'Right_Ankle',
             11:'Left_Hip',
             12:'Left_Knee',
             13:'Left_Ankle',
             14:'Chest',
             15:'Background'}

jointFrameMap = defaultdict(list)

# def adjustSpeed(frames, time): #may not be needed
#     newData = []
#     i = 0
#     l = len(frames)
    
#     a = int(time)
#     b = math.ceil(time)
#     print(a, b)
#     while i < l:
#         newData.append(frames[i])
#         if (i + a <= l) and (i + b) <= l:
#             spliceFrame = ((frames[int(i + a)], frames[int(i + b)]))
#             newData.append(
#                 tuple(map(lambda y: sum(y) / float(len(y)), zip(*spliceFrame))))
            
#         i += int(time * 2)
        
#     return newData


points1 = [(347, 125)]
points2 = [(356, 129)]
points3 = [(378, 137)]
points4 = [(323, 138)]
allPoints = [points1, points2, points3, points4]

def congregate(points):
    for i in range(len(points[0])):
        for j in range(len(points)):
            jointFrameMap[jointCode[i]].append(points[j][i])
            
congregate(allPoints)


In [11]:

codeMap = {
    (1, 0): 0,
    (1, 1): 1,
    (0, 1): 2,
    (-1, 1): 3,
    (-1, 0): 4,
    (-1, -1): 5,
    (0, -1): 6,
    (1, -1): 7
}
# This function generates the chaincode
# for transition between two neighbour points
''' 
dx dy    chain  direction
1  0       0        right
1  1       1        upper-right
0  1       2        up
-1 1       3        upper-left
-1 0       4        left
-1 -1      5        lower-left
0 -1       6        down
1 -1       7        lower-right
'''

def getChainCode(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return codeMap[(dx, dy)]


'''This function generates the list of
chaincodes for given list of points'''


def generateChainCode(linePoints):
    chainCode = []
    for i in range(len(linePoints) - 1):
        a = linePoints[i]
        b = linePoints[i + 1]
        chainCode.append(getChainCode(a[0], a[1], b[0], b[1]))
    return chainCode


def applyBresenham(x1, y1, x2, y2):
    linePoints = []
    linePoints.append([x1, y1])
    xdif = x2 - x1
    ydif = y2 - y1
    dx = abs(xdif)
    dy = abs(ydif)
    if(xdif > 0):
        xs = 1
    else:         
        xs = -1
    if (ydif > 0): 
        ys = 1
    else:          
        ys = -1
    if (dx > dy):

        d = 2 * dy - dx
        while (x1 != x2):
            x1 += xs
            if (d >= 0):
                y1 += ys
                d -= 2 * dx
            d += 2 * dy
            linePoints.append([x1, y1])
    else:

        d = 2 * dx-dy
        while(y1 != y2):
            y1 += ys
            if (d >= 0):
                x1 += xs
                d -= 2 * dy
            d += 2 * dx
            linePoints.append([x1, y1])
    return linePoints
 
 
def test():
    (x1, y1) = (347, 125)
    (x2, y2) = (356, 129)
    linePoints = applyBresenham(x1, y1, x2, y2)
    chainCode = generateChainCode(linePoints)
    chainCodeString = "".join(str(x) for x in chainCode)
    print ('Chain code for the straight line from', (x1, y1),
            'to', (x2, y2), 'is', chainCodeString)
    
test()    

'''
3 2 1 
4   0
5 6 7

'''
#010101010

Chain code for the straight line from (347, 125) to (356, 129) is 010101010


'\n3 2 1 \n4   0\n5 6 7\n\n'

In [20]:
def fullTest():
    for joint in jointFrameMap:
        print(joint)
        jointPoints = jointFrameMap[joint]
        print(jointPoints)
        
        totalChain = []
        for i in range(1, len(jointPoints)):
            p1 = jointPoints[i - 1]
            p2 = jointPoints[i]
            chainCode = ""
            linePoints = applyBresenham(p1[0], p1[1], p2[0], p2[1])
            chainCode = generateChainCode(linePoints)
            print(chainCode)
            totalChain.append("".join(str(x) for x in chainCode))
            print(totalChain)
            
fullTest()
            

Head
[(347, 125), (356, 129), (378, 137), (323, 138)]
[0, 1, 0, 1, 0, 1, 0, 1, 0]
['010101010']
[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]
['010101010', '0100101001001001010010']
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
['010101010', '0100101001001001010010', '4444444444444444444444444443444444444444444444444444444']
